In [35]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

In [2]:
DATA_DIR = '../_data/eCommerce-cosmetics/'
DATA_OCT = DATA_DIR+'2019-Oct.csv'
DATA_NOV = DATA_DIR+'2019-Nov.csv'
USECOLS = ["event_time","event_type","product_id","category_id","user_id"]
EVENT_THRESHOLD = 

In [3]:
df = pd.concat([pd.read_csv(DATA_OCT, engine='c', sep=',',usecols=USECOLS)
                ,pd.read_csv(DATA_NOV, engine='c', sep=',',usecols=USECOLS)])

In [4]:
df["event_type"] = df["event_type"].astype("category")
df['event_time'] = pd.to_datetime(df['event_time'],infer_datetime_format=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8738120 entries, 0 to 4635836
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   event_time   datetime64[ns]
 1   event_type   category      
 2   product_id   int64         
 3   category_id  int64         
 4   user_id      int64         
dtypes: category(1), datetime64[ns](1), int64(3)
memory usage: 341.7 MB


In [6]:
df

,event_time,event_type,product_id,category_id,user_id
0,2019-10-01 00:00:00,cart,5773203,1487580005134238553,463240011
1,2019-10-01 00:00:03,cart,5773353,1487580005134238553,463240011
2,2019-10-01 00:00:07,cart,5881589,2151191071051219817,429681830
3,2019-10-01 00:00:07,cart,5723490,1487580005134238553,463240011
4,2019-10-01 00:00:15,cart,5881449,1487580013522845895,429681830
...,...,...,...,...,...
4635832,2019-11-30 23:59:46,view,5830317,1487580009496313889,457678989
4635833,2019-11-30 23:59:47,view,5733064,1487580004832248652,422196217
4635834,2019-11-30 23:59:47,view,5867785,1487580007835370453,572579084
4635835,2019-11-30 23:59:57,view,5779406,2151191071051219817,540006764


In [7]:
def np_array_from_df(df):
    total_users = df[USER_ID].unique().shape[0]
    total_features = df[FEATURE_ID].unique().shape[0]
    print(total_users)
    print(total_features)
    interactions = np.zeros((total_users, total_features))
    i = 0
    for row in df.itertuples():
        if (i < 5) or (i == 495):
            print('{0:>3} | {1:>3} | {2:>1}'.format(row[6],row[4],row[5]))
        i +=1
        interactions[row[6], row[4]] = row[5]           
    return interactions

In [40]:
new_user_id = pd.DataFrame(df.user_id.unique())
len(new_user_id)

713100

In [41]:
new_product_id = pd.DataFrame(df.product_id.unique())
len(new_product_id)

45960

In [38]:
def user_experience_matrix(df):
    number_of_users = df['user_id'].unique().shape[0]
    number_of_features = df['product_id'].unique().shape[0]
    print(f"Number of unique users: {number_of_users}")
    print(f"Number of unique features (products for e-commerce): {number_of_features }")
    # Use np.float32 for torch.cuda.FloatTensor.or np.float16 for torch.cuda.HalfTensor (float64 not recommended)
    uxm = sp.dok_matrix((number_of_users, number_of_features), dtype=np.float32) 
    i = 0
    for row in df.itertuples():        
        i += 1
        if i<6:
#             print(row)
            print(f"{row.Index} | {row.event_time} | {row.event_type} | {row.product_id} | {row.category_id} | {row.user_id}")
        else:
            print(i)
            break
        
    return uxm


In [39]:
uxm = user_experience_matrix(df)
uxm

Number of unique users: 713100
Number of unique features (products for e-commerce): 45960
0 | 2019-10-01 00:00:00 | cart | 5773203 | 1487580005134238553 | 463240011
1 | 2019-10-01 00:00:03 | cart | 5773353 | 1487580005134238553 | 463240011
2 | 2019-10-01 00:00:07 | cart | 5881589 | 2151191071051219817 | 429681830
3 | 2019-10-01 00:00:07 | cart | 5723490 | 1487580005134238553 | 463240011
4 | 2019-10-01 00:00:15 | cart | 5881449 | 1487580013522845895 | 429681830
6


<713100x45960 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Dictionary Of Keys format>